# 显示设备列表

Qt提供数据表格控件QTableView
显示数据库的查询结果，
其样式与Excel显示效果非常接近。

QTableView使用非常简单，绑定QSqlQueryModel，
执行查询语句返回数据集。
```cpp
#include <QSqlQueryModel>
#include <QTableView>
QTableView *view = new QTableView();

QSqlQueryModel model;
model.setQuery("SELECT * FROM device");

view->setModel(&model);
view->show();
```
↓↓↓ 绑定显示表格
![](image/qt_table_view.png)

根据是否在线的条件，我们可以把查询语句改成：
```sql
SELECT serial, now()-refresh<20 AS online
FROM device
```
当前时间减去上一次刷新的时间，是否在20秒以内。

查看在界面的效果。

# 病人列表
新建病人表
```sql
CREATE TABLE `medical_monitor`.`patient` (
  `id` INT auto_increment,
  `name` CHAR(5) NOT NULL,
  `sex` CHAR(1) NULL,
  PRIMARY KEY (`id`));
```

首先，在数据库新建若干行假数据。
与设备列表类似，显示所有病人的列表。
## 新建病人
弹出一个对话框，填写参数，
按确定后执行SQL语句。

## 修改病人
如何修改某个病人的数据？
Qt已经给了丰富的接口，首先继承QSqlQueryModel
```cpp
class PatientModel: public QSqlQueryModel
{
public:
    PatientModel()
    {
        this->setQuery("SELECT * FROM patient");
    }
}

PatientModel patients;
QTableView *view = new QTableView();
view->setModel(&patients);
```
如此，应可以显示所有病人。

在QTableView负责显示控件时，我们需要告诉它，每个单元格是否可以编辑。
方法是重载flags函数。Qt在显示表格的每个单元格时，都会调用此函数，
以期获取该格子能否被编辑。
```cpp
Qt::ItemFlags flags(const QModelIndex &index) const override
{
    Qt::ItemFlags flags = QSqlQueryModel::flags(index);
    if (index.column() == 1 || index.column() == 2)
        flags |= Qt::ItemIsEditable;
    return flags;
}
```
表格的第2列（姓名）和第3列（性别），可以编辑。

此时，编辑似乎无效。我们还需要响应编辑完成后的事件。
Qt也有标准方法：重载setData函数。
```cpp
bool setData(const QModelIndex &index, const QVariant &value, int /* role */)
{
    if (index.column() < 1 || index.column() > 2)
        return false;

    // index.row() 返回被编辑的行
    // 第二个参数0，代表第一列
    // 这里是获取一个单元格位置的对象
    QModelIndex primaryKeyIndex = QSqlQueryModel::index(index.row(), 0);
    
    // 该单元格的值
    int id = this->data(primaryKeyIndex).toInt();

    bool ok;
    // 修改第2列的单元格，设置姓名
    if (index.column() == 1) {
        QSqlQuery query;
        query.prepare("update patient set name = ? where id = ?");
        query.addBindValue(name);
        query.addBindValue(id);
        ok =  query.exec();
    }
    return ok;
}
```
执行完成，我们发现界面没变化。
但其实我们直接用工具查看数据表，发现数据已经变了。
其原因是，我们在setData的时候，执行了update语句，更新了数据库，
却并未把最新的数据反映到界面上，
所以需要重新加载一遍数据。
```cpp
void refresh()
{
    this->setQuery("SELECT * FROM patient");
}

//setData 返回之前调用refresh
```

# 病人与设备关联
病人与设备是多对多的关系，因此需建立关联表。
```sql
CREATE TABLE `medical_monitor`.`device_patient` (
  `dev_id` INT NOT NULL,
  `patient_id` INT NOT NULL,
  `time` TIMESTAMP DEFAULT NOW(),
  CONSTRAINT `fk_device`
    FOREIGN KEY (`dev_id`)
    REFERENCES `medical_monitor`.`device` (`dev_id`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_patient`
    FOREIGN KEY (`patient_id`)
    REFERENCES `medical_monitor`.`patient` (`id`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION);
```

三个字段：
- 设备的外键，引用设备号
- 病人外键
- 病人与设备绑定的时间，默认值是NOW，即输入行的时间

不需要解绑定。设备同时只能给一个病人使用，
所以同一设备当中时间最新的一行，就是当前在用的病人。

## 绑定关系
界面显示病人的列表，需要增加一个列，代表当前他所对应的设备。
首先准备数据，在数据库手工创建一些绑定关系。

```sql
select * 
from patient, device, device_patient
where patient.id = device_patient.patient_id
    and device.dev_id = device_patient.dev_id
```
三个表联合查询，利用关系。
上述语句有问题，如果病人未绑定任何设备，则不显示。
需用LEFT JOIN解决。

```sql
select * from patient
left join device_patient
on patient.id = device_patient.patient_id
left join device
on device.dev_id = device_patient.dev_id
```
解释LEFT JOIN：

注意：三个表交叉后，* 代表的字段太多，可以显示指定有用的字段。

## 为病人选择设备
选中病人表当中的某一行，点击界面上“绑定设备”的按钮，
弹出来对话框，显示所有机器的单选框，
请用户选择绑定的机器号。

```cpp
// 支持同时选择多行，所以返回的是选择的列表
QModelIndexList selection = view->selectionModel()->selectedRows();

if(selection.count() > 0)
{
    QModelIndex index = selection.at(0);
    qDebug() << index.row();
}
```